In [ ]:
# import our data creation tool
# idk how to import a function from anotheripynb
from data_exploration/create_data.ipynb import process_all_tracks

TRACKS_ROOT = "./data/assetto_corsa_tracks"
OUTPUT_ROOT = "./data/extracted_track_data"
process_all_tracks(TRACKS_ROOT, OUTPUT_ROOT)